In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

In [2]:
# Criação de uma SparkSession com configuração para usar o Hive
spark = SparkSession.builder \
    .appName("Test Hive Metastore") \
    .enableHiveSupport() \
    .getOrCreate()



In [3]:
def ingest(spark, projeto, table, format, schema=None):
    """
    projeto == database
    """
    raw_path = f"s3a://raw/{projeto}/{table}/*"
    bronze_path = f"s3a://data/bronze/{projeto}/{table}"
    print(1)    
    spark.sql(f"CREATE DATABASE IF NOT EXISTS bronze_{projeto}")
    
    print(2)   
    df = spark.read.format(format).option('header', 'true').schema(schema).load(raw_path)
    
    print(3)   
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS bronze_{projeto}.{table} 
        USING {format} 
        LOCATION '{bronze_path}'
    """)
    
    print(4)   
    df.write.mode("overwrite").format("parquet").saveAsTable(f"{projeto}.{table}")


In [4]:
arqschema = "Data STRING, Valor DOUBLE, Identificador STRING, Descricao STRING"
# arqschema = "date DATE, title STRING, amount DOUBLE"
# ingest(spark,'nubank','extrato','csv',arqschema)

In [8]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|     test|
+---------+



In [7]:
spark.sql("CREATE DATABASE TEST").show()

++
||
++
++



In [7]:
df = spark.read.format("csv").option('header', 'true').schema(arqschema).load("s3a://raw/nubank/extrato/")
df.show()


+----------+--------+--------------------+--------------------+
|      Data|   Valor|       Identificador|           Descricao|
+----------+--------+--------------------+--------------------+
|01/12/2023|   -90.0|656a0c5d-1bc3-456...|Transferência env...|
|01/12/2023|    45.0|656a0c5d-7e49-4df...|Transferência Rec...|
|01/12/2023|   -15.0|656a8fcb-514e-483...|Compra no débito ...|
|01/12/2023|    15.0|656a90f6-a652-4c7...|Transferência Rec...|
|02/12/2023|   -20.0|656b8142-e8de-48c...|Transferência env...|
|02/12/2023|   -15.0|656b8b9e-8a84-462...|Transferência env...|
|05/12/2023|    50.0|656f1b0a-3913-454...|Transferência Rec...|
|06/12/2023|  139.49|65703a1c-89e3-4fa...|Transferência Rec...|
|06/12/2023|  720.99|6570c32e-cb5f-46e...|Transferência Rec...|
|06/12/2023|  -240.0|6570c930-22f7-4f1...|Transferência env...|
|06/12/2023|-1107.39|6570ca14-2a34-43d...| Pagamento de fatura|
|06/12/2023|  -108.9|6571170d-b1da-4b7...|Transferência env...|
|07/12/2023|    31.0|657247be-f83a-413..

In [8]:
df.write.mode("overwrite").format("parquet").saveAsTable("default.extratos")

Py4JJavaError: An error occurred while calling o58.saveAsTable.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.validateTableLocation(SessionCatalog.scala:411)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:176)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution

In [ ]:
def ingest(spark, projeto, table, format, schema=None):
    raw = f"s3a://raw/{projeto}/{table}/*"
    bronze = f"s3a://data/bronze/{projeto}/{table}"

    df = spark.read.format(format).option('header','true').schema(schema).load(raw)
    df.write.mode('overwrite').save(bronze)